In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed

from IPython.core.debugger import set_trace

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import time
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import wordnet as wn
from collections import defaultdict
from collections import OrderedDict
from collections import Counter
from itertools import chain
import string
import re
import unidecode
import spacy
from sklearn.feature_extraction.text import CountVectorizer

from wordcloud import WordCloud

plt.style.use(style="seaborn")
%matplotlib inline


In [2]:
tuites = pd.read_csv("../input/textos-2020-tweets-pt-covid-19/textos_tuitesPt_2020.csv")
tuites.drop_duplicates(subset=['text'], inplace=True)
tuites.reset_index(drop=True, inplace=True)
tuites["id"] = tuites.index
tuites

,text,id
0,@drayareturn coronavirus aceita a todos de bra...,0
1,Acho que os eleitores do Bolsonaro não deveria...,1
2,Ainda temos dengue..Zika..hn1..sarampo... fome...,2
3,trancar todos os stans da c***** dos replies d...,3
4,1 caso de coronavirus confirmado no Brasil e t...,4
...,...,...
5273475,OMS pede alerta para o aumento do coronavírus ...,5273475
5273476,Twitter vai desativar conta que desrespeitar 5...,5273476
5273477,#JN De os nomes dos lideres então que vamos di...,5273477
5273478,Putz https://t.co/nCsVzX7Dav,5273478


In [3]:
# Acessa diretamente algum dos textos

A = 20000
print(tuites['text'][A])

Coronavirus não é nada perto desse povo kkkkkkk https://t.co/PdgE9O7il5


In [4]:
#printa os X primeiros registros do dataframe

X = 10
tuites.head(X)

,text,id
0,@drayareturn coronavirus aceita a todos de bra...,0
1,Acho que os eleitores do Bolsonaro não deveria...,1
2,Ainda temos dengue..Zika..hn1..sarampo... fome...,2
3,trancar todos os stans da c***** dos replies d...,3
4,1 caso de coronavirus confirmado no Brasil e t...,4
5,Luís Sepúlveda sentiu os primeiros sintomas de...,5
6,1 caso de coronavirus confirmado no Brasil e t...,6
7,Luxemburgo confirmou seu primeiro caso de #cor...,7
8,Eu: num auditório na Póvoa \nExpresso: ALERTA ...,8
9,"E o pior é que eu queria tanto aaaaaa, pneumon...",9


In [5]:
# uma lista personalizada de stop words, mais adaptada para a realidade das mídias sociais

extra_sw = ['pfv', 'mto', 'ia', 'oq', 'msm', 'tão', 'enquanto', 'porque', 'nesta', 'mim', 'disso', 'quanto', 'tava', 'nessa', 'ja', 'tanto', 'qualquer', 'lá', 'pois', 'dessa', 'nesse', 'toda', 'vcs', 'dá', 'pra', 'tá', 'ta', 'pq', 'vc', 'aí', 'assim', 'pro', 'tô', 'to', 'desse', 'né', 'então', 'ai', 'a', 'à', 'agora', 'ainda', 'alguém', 'algum', 'alguma', 'algumas', 'alguns', 'ampla', 'amplas', 'amplo', 'amplos', 'ante', 'antes', 'ao', 'aos', 'após', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'até', 'através', 'cada', 'coisa', 'coisas', 'com', 'como', 'contra', 'contudo', 'da', 'daquele', 'daqueles', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'dessa', 'dessas', 'desse', 'desses', 'desta', 'destas', 'deste', 'deste', 'destes', 'deve', 'devem', 'devendo', 'dever', 'deverá', 'deverão', 'deveria', 'deveriam', 'devia', 'deviam', 'disse', 'disso', 'disto', 'dito', 'diz', 'dizem', 'do', 'dos', 'e', 'é', "e'", 'ela', 'elas', 'ele', 'eles', 'em', 'enquanto', 'entre', 'era', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'estes', 'estou', 'eu', 'fazendo', 'fazer', 'feita', 'feitas', 'feito', 'feitos', 'foi', 'for', 'foram', 'fosse', 'fossem', 'grande', 'grandes', 'há', 'isso', 'isto', 'já', 'la', 'la', 'lá', 'lhe', 'lhes', 'lo', 'mas', 'me', 'mesma', 'mesmas', 'mesmo', 'mesmos', 'meu', 'meus', 'minha', 'minhas', 'muita', 'muitas', 'muito', 'muitos', 'na', 'não', 'nas', 'nem', 'nenhum', 'nessa', 'nessas', 'nesta', 'nestas', 'ninguém', 'no', 'nos', 'nós', 'nossa', 'nossas', 'nosso', 'nossos', 'num', 'numa', 'nunca', 'o', 'os', 'ou', 'outra', 'outras', 'outro', 'outros', 'para', 'pela', 'pelas', 'pelo', 'pelos', 'pequena', 'pequenas', 'pequeno', 'pequenos', 'per', 'perante', 'pode', 'pôde', 'podendo', 'poder', 'poderia', 'poderiam', 'podia', 'podiam', 'pois', 'por', 'porém', 'porque', 'posso', 'pouca', 'poucas', 'pouco', 'poucos', 'primeiro', 'primeiros', 'própria', 'próprias', 'próprio', 'próprios', 'quais', 'qual', 'quando', 'quanto', 'quantos', 'que', 'quem', 'são', 'se', 'seja', 'sejam', 'sem', 'sempre', 'sendo', 'será', 'serão', 'seu', 'seus', 'si', 'sido', 'só', 'sob', 'sobre', 'sua', 'suas', 'talvez', 'também', 'tampouco', 'te', 'tem', 'tendo', 'tenha', 'ter', 'teu', 'teus', 'ti', 'tido', 'tinha', 'tinham', 'toda', 'todas', 'todavia', 'todo', 'todos', 'tu', 'tua', 'tuas', 'tudo', 'última', 'últimas', 'último', 'últimos', 'um', 'uma', 'umas', 'uns', 'vendo', 'ver', 'vez', 'vindo', 'vir', 'vos', 'vós']
supeitas_sw = ['todo', 'todos', 'toda', 'todas', 'todxs', 'todes', 'apenas', 'sobre', 'outro', 'desde', 'muita', 'quase', 'segundo', 'outros', 'cada', 'alguém', 'ainda', 'aqui', 'devido', 'enquanto', 'onde']
verbos_sw = ['sendo', 'vão', 'quer', 'vamos', 'pode', 'ser', 'queria', 'deu', 'quero', 'ir', 'dar', 'ter', 'via', 'disse', 'fez', 'vem', 'sei', 'podem', 'vai', 'vou']

# importa a lista de stopwords em português da lib nltk
# armazenando a lista de stopwords em uma variável

stop = set(stopwords.words("portuguese"))
stop.update(extra_sw)
stop.update(supeitas_sw)
stop.update(verbos_sw)
'''
Adicionar em um set:
set.add(1)         # concatena o elemento no set (se colocar uma lista aqui, quem entra é a lista e não os elementos individuais dela)
set.update([1,2])  # concatena os elementos de uma tupla, set, lista ou dicionário no set
'''

'\nAdicionar em um set:\nset.add(1)         # concatena o elemento no set (se colocar uma lista aqui, quem entra é a lista e não os elementos individuais dela)\nset.update([1,2])  # concatena os elementos de uma tupla, set, lista ou dicionário no set\n'

In [6]:
# Função para remover URLs
# utiliza expressões regulares para encontrar características de url, html e menções no texto
def remover_URL(texto):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", texto)

# Função para remover menções a usuários (ex: @joao, @maria)
def remover_mencao(texto):
    #mencao = re.compile(r"@\w+")
    return re.sub(r"(?:\@|https?\://)\S+", "", texto)

# Função para remover códigos e notações HTML
def remover_html(texto):
    html = re.compile(r"<.*?>")
    return html.sub(r"", texto)

# Função para remover emojis
def remover_emoji(string): # busca pela representação UNICODE de emojis e outros símbolos
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F" # emoticons
        u"\U0001F300-\U0001F5FF" # símbolos e pictogramas
        u"\U0001F680-\U0001F6FF" # transporte e símbolos de mapas
        u"\U0001F1E0-\U0001F1FF" # bandeiras (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)

# Função para remover pontuação
def remover_pontuacao(texto):
    tabela = str.maketrans("", "", string.punctuation)
    return texto.translate(tabela)

# Função para remover todos os não-alfanuméricos
def remover_nao_alfanum(texto):
    return re.sub(r'[^\w]', ' ', texto)

# Função para remover números
def remover_numeros(texto):
    return re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", texto)

# Função para remover as stopwords
def remover_stopwords(texto):
    texto = [palavra.lower() for palavra in texto.split() if palavra.lower() not in stop]
    return " ".join(texto)

# Função para remover acentos de um texto
def remover_acentos(text):
    return unidecode.unidecode(text) # se der erro, converter text para string (str(text))

# Função para lematização de textos
def lematizar(textos):
    docs = nlp.pipe(textos)
    lemmas_list = [" ".join([token.lemma_ for token in doc]) for doc in docs]
    return lemmas_list

# Remove caracteres duplicados (ex: kkkkkkkkk) e caracteres sozinhos (ex: q),
# exceto números e duplas de "r" e "s"
def remover_sequencia(texto):
    padrao = r'([^rs\d])(\1{1,})'
    subst = r'\1'
    return re.sub(padrao, subst, texto)

# Remove palavras com menos de 3 letras
def remover_sozinho(texto):
    texto = [w for w in texto.split() if len(w)>2]
    return ' '.join(texto)

# Função que remove palavras raras, definidas por um limiar
def remover_token_raro(textos):

    # divide o corpus em palavras
    v = textos.str.split().tolist() # [s.split() for s in tuites2["text"].tolist()]

    # calcula a frequência global das palavras
    c = Counter(chain.from_iterable(v))

    # filtra as palavras com frequência menor do que um limiar
    limiar = 11 # valor pode ser ajustado livremente
    v = [[w for w in s if c[w] >= limiar] for s in v]

    # junta tudo e reatribui
    return [' '.join(s) for s in v]

# Função que remove palavras que se repetem em x% dos registros
def remover_token_repetido(textos, x=50):
    
    # Conta a frequência das palavras na coluna texto
    freq = textos.str.split(expand=True).stack().value_counts()
    
    # Filtra as palavras que aparecem em pelo menos x% das linhas
    palavras = freq[freq >= x/100 * len(textos)].index.tolist()
    
    # Aplica a função em cada linha do dataframe
    return textos.apply(lambda s: " ".join([w for w in s.split() if w not in palavras]))

In [7]:
#Baixando o modelo do português para lematizar
spacy.cli.download("pt_core_news_sm")

#Carregando o modelo do português
nlp = spacy.load("pt_core_news_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 41.8 MB/s eta 0:00:00


✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [8]:
# executa as funções para "limpar" o texto de elementos desnecessários:
'''
São removidas nessa ordem: URLs, menções(ex: @joao), elementos HTML, emojis, pontuação, não-alfanumericos, números,
stopwords, remover acentuação, caracteres em sequência(ex: kkkkkkk), caracteres sozinhos, textos que estejam duplicados
depois de tudo isso, e palavras repetidas no mesmo registro
'''

tuites["text"] = tuites.text.map(lambda x: remover_URL(x))
tuites["text"] = tuites.text.map(lambda x: remover_mencao(x))
tuites["text"] = tuites.text.map(lambda x: remover_html(x))
tuites["text"] = tuites.text.map(lambda x: remover_emoji(x))
tuites["text"] = tuites.text.map(lambda x: remover_pontuacao(x))
tuites["text"] = tuites.text.map(lambda x: remover_nao_alfanum(x))
tuites["text"] = tuites.text.map(lambda x: remover_numeros(x))

In [9]:
# Pré-processamento, parte 2. Separados para melhor organização do código

tuites["text"] = tuites["text"].map(remover_stopwords)
tuites['text'] = tuites['text'].apply(remover_acentos)
tuites["text"] = tuites.text.map(lambda x: remover_sequencia(x))
tuites["text"] = lematizar(tuites["text"]) # método mais rápido de lematização (paralelização)
#tuites["text"] = tuites.text.map(lambda x: lematizar(x)) # método MUITO lento de lematização

In [10]:
# Pré-processamento, parte 3

tuites["text"] = tuites["text"].map(remover_sozinho)
tuites["text"] = (tuites["text"].str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' ')) # remove palavras duplicadas no mesmo documento

In [11]:
#Pré-processamento, parte 4

tuites["text"] = remover_token_repetido(tuites["text"], 50)

In [12]:
#remove os registros que tenham ficado iguais depois do pré-processamento

tuites.drop_duplicates(subset=['text'], inplace=True)
tuites.reset_index(drop=True, inplace=True)
tuites["id"] = tuites.index
tuites

,text,id
0,coronavirus aceitar braco abrir,0
1,achar eleitor bolsonaro medo coronavirus febre...,1
2,denguezikahn1sarampo fome coronavirus entrar fila,2
3,trancar stans replies twetr sala coronavirus,3
4,caso coronavirus confirmar Brasil mundo querer...,4
...,...,...
4010758,parabem lmportante iniciativa curitibano tea f...,4010758
4010759,brumar confirmar 62a morte coronavirus pacient...,4010759
4010760,oms pedir alertar aumento coronavirus chine ap...,4010760
4010761,nome lider discutir erro ocorrer luta covid19 ...,4010761


In [13]:
# remover palavras que aparecem com pouca frequência
tuites["text"] = remover_token_raro(tuites["text"])

In [14]:
# remove os registros que tenham ficado iguais depois do pré-processamento

tuites.drop_duplicates(subset=['text'], inplace=True)
tuites.reset_index(drop=True, inplace=True)
tuites["id"] = tuites.index
tuites

,text,id
0,coronavirus aceitar braco abrir,0
1,achar eleitor bolsonaro medo coronavirus febre...,1
2,fome coronavirus entrar fila,2
3,trancar replies twetr sala coronavirus,3
4,caso coronavirus confirmar Brasil mundo querer...,4
...,...,...
3941029,parabem iniciativa curitibano tea familia,3941029
3941030,brumar confirmar 62a morte coronavirus pacient...,3941030
3941031,oms pedir alertar aumento coronavirus chine ap...,3941031
3941032,nome lider discutir erro ocorrer luta covid19 ...,3941032


In [15]:
#função que retorna os top nr n-gramas no corpus
def get_top_text_ngrams(corpus, ngramas=(1, 1), nr=None):
    """
    Cria um bag de n-gramas e conta a frequência dos n-gramas
    Retorna um lista ordenada de tuplas: (n-grama, contador)
    
    O primeiro parâmentro é o corpus. Esse método aceita o corpus como um todo, o anterior aceita apenas um documento de cada vez.
    O segundo parâmetro deve ser um tupla. (1,1) para unigrmas, (2,2) para bigramas e assim por diante.
    O terceiro parâmetro é um inteiro, o número de top n-gramas que deseja-se retornar.
    """
    vec = CountVectorizer(ngram_range=ngramas).fit(corpus)
    bag_of_words = vec.transform(corpus)
    soma_palavras = bag_of_words.sum(axis=0)
    palavras_freq = [(palavra, soma_palavras[0, idx]) for palavra, idx in vec.vocabulary_.items()]
    palavras_freq = sorted(palavras_freq, key=lambda x: x[1], reverse=True)

    return palavras_freq[:nr]

In [16]:
top_unigramas_texto = get_top_text_ngrams(tuites.text, ngramas=(1, 1), nr=-50)
top_unigramas_texto

[('covid19', 1836104),
 ('coronavirus', 903223),
 ('caso', 323137),
 ('brasil', 301052),
 ('morte', 239528),
 ('novo', 223368),
 ('dia', 197277),
 ('pessoa', 195555),
 ('saude', 193550),
 ('pandemia', 180073),
 ('bolsonaro', 138918),
 ('morrer', 132641),
 ('mil', 125330),
 ('governo', 125212),
 ('bom', 125019),
 ('ano', 123947),
 ('falar', 121774),
 ('ficar', 120984),
 ('mundo', 120048),
 ('saber', 116582),
 ('pai', 112782),
 ('numero', 112088),
 ('gente', 109817),
 ('vacinar', 109430),
 ('presidente', 101673),
 ('confirmar', 101415),
 ('virus', 98849),
 ('chegar', 97267),
 ('hoje', 97106),
 ('casa', 89095),
 ('fazer', 88656),
 ('teste', 88203),
 ('registrar', 87878),
 ('covid', 86531),
 ('passar', 84812),
 ('brasileiro', 82443),
 ('vacina', 81784),
 ('obito', 81476),
 ('positivo', 80439),
 ('achar', 78388),
 ('bem', 77929),
 ('hospital', 77315),
 ('tomar', 75279),
 ('vida', 74416),
 ('ver', 73905),
 ('vacino', 71221),
 ('paciente', 70017),
 ('povo', 67777),
 ('usar', 66489),
 ('deus',

In [17]:
#função para ordenar a lista consoante ao segundo item da tupla
def myFunc(e):
    return e[1]
top_unigramas_texto.sort(key=myFunc)
top_unigramas_texto

[('oscurecer', 7),
 ('naol', 7),
 ('1891', 7),
 ('apropo', 7),
 ('escolter', 8),
 ('preneu', 8),
 ('0808', 8),
 ('prenem', 8),
 ('0107', 8),
 ('veramente', 8),
 ('keim', 8),
 ('2328', 8),
 ('0208', 8),
 ('lesgo', 8),
 ('ditelo', 8),
 ('sdj', 8),
 ('030', 8),
 ('texturo', 8),
 ('felic', 8),
 ('004', 8),
 ('2906', 8),
 ('carryon', 8),
 ('coronavirusexplained', 8),
 ('healthportugal', 8),
 ('reydelmixeochalenge', 8),
 ('chrizlakruz', 8),
 ('1047', 8),
 ('chequear', 8),
 ('2807', 8),
 ('cryptocurrencie', 8),
 ('2806', 8),
 ('2097', 8),
 ('3262', 8),
 ('indvsau', 8),
 ('binanceresolution', 8),
 ('hdelacruzo', 9),
 ('conmebolibertador', 9),
 ('hosano', 9),
 ('apesto', 9),
 ('bombay', 9),
 ('pedorro', 9),
 ('2306', 9),
 ('hshshshshs', 9),
 ('irrelevant', 9),
 ('comentaris', 9),
 ('narcissist', 9),
 ('afei', 9),
 ('decisamente', 9),
 ('olvidel', 9),
 ('reloj', 9),
 ('769', 9),
 ('inhumanity', 9),
 ('uoc', 9),
 ('envejar', 9),
 ('batho', 9),
 ('epicgames', 9),
 ('kha', 9),
 ('sicuramente', 9),


In [18]:
len(top_unigramas_texto)

70303

In [19]:
tuites['text'].to_csv('(processado)textos_tuitesPt_2020.csv.gz', index=False, header=False, compression='gzip')